## Standard Scalar

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pandas as pd

#categorizing the input and output data
bat_data = pd.read_csv('NEWTrainingData_StandardScaler.csv')
train_bat = bat_data

D_in, H, H2, D_out, N = 165, 100, 75, 2, 4000
dtype = torch.float
device = torch.device('cpu')


#define testing sets
x_train = train_bat.drop(columns=['Unnamed: 0', 'Gravimetric Capacity (units)', 'Volumetric Capacity', 'Max Delta Volume'])
y_train = train_bat[['Gravimetric Capacity (units)', 'Volumetric Capacity']]

#test-train split
#shuffle
x_train = x_train.sample(frac = 1)
y_train = y_train.sample(frac = 1)

#optimize for user data
x_test = x_train[4000:]
y_test = y_train[4000:]

x_train = x_train[:4000]
y_train = y_train[:4000]

#Defining training and testing data
x_train_np = np.array(x_train)
x_train_torch = torch.tensor(x_train_np, device = device, dtype = dtype)

y_train_np = np.array(y_train)
y_train_torch = torch.tensor(y_train_np, device = device, dtype = dtype)

x_test_np = np.array(x_test)
x_test_torch = torch.tensor(x_test_np, device = device, dtype = dtype)

y_test_np = np.array(y_test)
y_test_torch = torch.tensor(y_test_np, device = device, dtype = dtype)

#Defining weights
w1 = torch.randn(D_in, H, device=device, dtype=dtype)
w2 = torch.randn(H, D_out, device=device, dtype=dtype)
#w3 = torch.randn(H2, D_out, device=device, dtype=dtype)

#define model
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.LeakyReLU(),
    torch.nn.Linear(H, H2),
    torch.nn.LeakyReLU(),
    torch.nn.Linear(H2, D_out),
    #nn.Softmax(dim=1) #normalizing the data,
)

optimizer = optim.SGD(model.parameters(), lr=1e-5)
loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-5
for t in range(1000):
    # Forward pass: compute predicted y by passing x to the model. Module objects
    # override the __call__ operator so you can call them like functions. When
    # doing so you pass a Tensor of input data to the Module and it produces
    # a Tensor of output data.
    y_pred = model(x_train_torch)

    # Compute and print loss. We pass Tensors containing the predicted and true
    # values of y, and the loss function returns a Tensor containing the
    # loss.
    loss = loss_fn(y_pred, y_train_torch)
    if t % 100 == 99:
        print(t, loss.item())

    # Zero the gradients before running the backward/training pass.
    optimizer.zero_grad()
    model.zero_grad()

    # Backward pass: compute gradient of the loss with respect to all the learnable
    # parameters of the model. Internally, the parameters of each Module are stored
    # in Tensors with requires_grad=True, so this call will compute gradients for
    # all learnable parameters in the model.
    loss.backward()
    optimizer.step()

    # Update the weights using gradient descent. Each parameter is a Tensor, so
    # we can access its gradients like we did before.
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad


99 8075.4755859375
199 7764.99560546875
299 6988.41943359375
399 6006.9443359375
499 5055.3681640625
599 4557.8203125
699 4773.828125
799 3752.041259765625
899 4044.66015625
999 3387.39501953125


### Test Error

In [3]:
y_pred_test = model(x_test_torch)
loss_fn(y_pred_test, y_test_torch)

tensor(532.8734, grad_fn=<MseLossBackward>)

### RSS

In [5]:
RSS = ((model(x_train_torch)-y_train_torch)**2).sum()
print(RSS)

tensor(3915.1335, grad_fn=<SumBackward0>)


## MinMax Scalar

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pandas as pd

#categorizing the input and output data
bat_data = pd.read_csv('NEWTrainingData_MinMaxScaler.csv')
train_bat = bat_data

D_in, H, H2, D_out, N = 115, 100, 75, 2, 4000
dtype = torch.float
device = torch.device('cpu')


#define testing sets
x_train = train_bat.drop(columns=['Unnamed: 0', 'Gravimetric Capacity (units)', 'Volumetric Capacity', 'Max Delta Volume'])
y_train = train_bat[['Gravimetric Capacity (units)', 'Volumetric Capacity']]

#shuffle data
x_train = x_train.sample(frac=1)
y_train = y_train.sample(frac=1)

#test-train split
#optimize for user data
x_test = x_train[4000:]
y_test = y_train[4000:]

x_train = x_train[:4000]
y_train = y_train[:4000]

#Defining training and testing data
x_train_np = np.array(x_train)
x_train_torch = torch.tensor(x_train_np, device = device, dtype = dtype)

y_train_np = np.array(y_train)
y_train_torch = torch.tensor(y_train_np, device = device, dtype = dtype)

x_test_np = np.array(x_test)
x_test_torch = torch.tensor(x_test_np, device = device, dtype = dtype)

y_test_np = np.array(y_test)
y_test_torch = torch.tensor(y_test_np, device = device, dtype = dtype)

#Defining weights
w1 = torch.randn(D_in, H, device=device, dtype=dtype)
w2 = torch.randn(H, D_out, device=device, dtype=dtype)
#w3 = torch.randn(H2, D_out, device=device, dtype=dtype)

#define model
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.LeakyReLU(),
    torch.nn.Linear(H, H2),
    torch.nn.LeakyReLU(),
    torch.nn.Linear(H2, D_out),
    #nn.Softmax(dim=1) #normalizing the data,
)

optimizer = optim.SGD(model.parameters(), lr=1e-4)
loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-4
for t in range(1000):
    # Forward pass: compute predicted y by passing x to the model. Module objects
    # override the __call__ operator so you can call them like functions. When
    # doing so you pass a Tensor of input data to the Module and it produces
    # a Tensor of output data.
    y_pred = model(x_train_torch)

    # Compute and print loss. We pass Tensors containing the predicted and true
    # values of y, and the loss function returns a Tensor containing the
    # loss.
    loss = loss_fn(y_pred, y_train_torch)
    if t % 100 == 99:
        print(t, loss.item())

    # Zero the gradients before running the backward/training pass.
    optimizer.zero_grad()
    model.zero_grad()

    # Backward pass: compute gradient of the loss with respect to all the learnable
    # parameters of the model. Internally, the parameters of each Module are stored
    # in Tensors with requires_grad=True, so this call will compute gradients for
    # all learnable parameters in the model.
    loss.backward()
    optimizer.step()

    # Update the weights using gradient descent. Each parameter is a Tensor, so
    # we can access its gradients like we did before.
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad


99 38.27134323120117
199 37.96364974975586
299 37.73960876464844
399 37.537025451660156
499 37.34566116333008
599 37.15192794799805
699 36.94499206542969
799 36.73301315307617
899 36.527015686035156
999 37.5709342956543


### Test Error

In [7]:
y_pred_test = model(x_test_torch)
loss_fn(y_test_torch, y_pred_test)

tensor(6.0370, grad_fn=<SumBackward0>)

### RSS

In [9]:
RSS = ((model(x_train_torch)-y_train_torch)**2).sum()
print(RSS)

tensor(37.6390, grad_fn=<SumBackward0>)


## Predicting Volume Change

In [10]:
train_bat

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,108,109,110,111,112,113,114,Gravimetric Capacity (units),Volumetric Capacity,Max Delta Volume
0,0,0.486246,0.060893,-0.214474,-0.112607,0.675571,0.298940,1.110048,-0.409336,-0.608829,...,0.013285,0.008151,-0.004783,0.001267,-0.005504,0.005775,0.002805,0.017651,0.032282,0.000107
1,1,0.694759,-1.063526,0.001798,-0.221694,-0.357235,0.335010,0.456398,-0.564281,-0.188967,...,0.006995,0.012646,0.007878,-0.006621,-0.010773,0.004906,0.009384,0.013516,0.017134,0.000066
2,2,0.679393,-0.770838,-0.507284,0.441374,0.449890,0.084205,0.534985,-0.494704,0.691186,...,0.013040,0.008098,-0.001060,-0.008740,-0.041023,-0.032473,-0.005981,0.069254,0.082636,0.000228
3,3,0.222118,-0.352762,-1.097785,0.364854,-0.194111,0.090228,0.444386,-0.425387,-0.030196,...,0.011135,0.010524,-0.000659,-0.004796,0.001783,0.001435,0.005299,0.026762,0.040243,0.000079
4,4,0.496687,-1.018132,0.039497,-0.122775,-0.261139,0.298638,0.410037,-0.529058,-0.159984,...,0.012249,0.011378,-0.004106,-0.003080,0.001072,-0.001411,0.004857,0.026762,0.037129,0.000095
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4396,4396,-0.108310,0.940409,0.959655,1.113824,-0.307627,-0.127549,-0.372501,0.182058,0.143321,...,0.013122,0.009733,-0.032890,-0.021991,0.034519,0.002598,-0.013542,0.034073,0.057571,0.000260
4397,4397,0.731422,0.527013,0.328726,0.594572,-0.452792,-0.342455,0.388687,0.537657,0.289902,...,-0.010045,-0.012350,0.003835,0.012596,0.004243,0.011012,0.009617,0.021788,0.047893,0.000024
4398,4398,0.522887,1.133211,-0.909830,-0.269775,-0.692468,0.249483,0.157622,0.250940,-0.109424,...,-0.005085,0.006921,0.001318,0.002289,-0.006700,-0.001908,-0.002679,0.051223,0.117200,0.000217
4399,4399,-0.297020,1.228747,-0.548483,-0.237261,-0.713716,-0.278450,0.154214,-0.274529,-0.121010,...,0.011201,0.015631,0.007458,-0.011007,-0.004750,-0.001337,-0.000638,0.041988,0.093676,0.000072


In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pandas as pd

#categorizing the input and output data
bat_data = pd.read_csv('NEWTrainingData_MinMaxScaler.csv')
train_bat = bat_data

D_in, H, H2, D_out, N = 2, 100, 75, 1, 4000
dtype = torch.float
device = torch.device('cpu')


#define testing sets
x_train = train_bat[['Gravimetric Capacity (units)', 'Volumetric Capacity']]
y_train = train_bat['Max Delta Volume']

#shuffle data
x_train = x_train.sample(frac=1)
y_train = y_train.sample(frac=1)

#test-train split
#optimize for user data
x_test = x_train[4000:]
y_test = y_train[4000:]

x_train = x_train[:4000]
y_train = y_train[:4000]

#Defining training and testing data
x_train_np = np.array(x_train)
x_train_torch = torch.tensor(x_train_np, device = device, dtype = dtype)

y_train_np = np.array(y_train)
y_train_torch = torch.tensor(y_train_np, device = device, dtype = dtype)

x_test_np = np.array(x_test)
x_test_torch = torch.tensor(x_test_np, device = device, dtype = dtype)

y_test_np = np.array(y_test)
y_test_torch = torch.tensor(y_test_np, device = device, dtype = dtype)

#Defining weights
w1 = torch.randn(D_in, H, device=device, dtype=dtype)
w2 = torch.randn(H, D_out, device=device, dtype=dtype)
#w3 = torch.randn(H2, D_out, device=device, dtype=dtype)

#define model
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.LeakyReLU(),
    torch.nn.Linear(H, H2),
    torch.nn.LeakyReLU(),
    torch.nn.Linear(H2, D_out),
    #nn.Softmax(dim=1) #normalizing the data,
)

optimizer = optim.SGD(model.parameters(), lr=1e-8)
loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-8
for t in range(1000):
    # Forward pass: compute predicted y by passing x to the model. Module objects
    # override the __call__ operator so you can call them like functions. When
    # doing so you pass a Tensor of input data to the Module and it produces
    # a Tensor of output data.
    y_pred = model(x_train_torch)

    # Compute and print loss. We pass Tensors containing the predicted and true
    # values of y, and the loss function returns a Tensor containing the
    # loss.
    loss = loss_fn(y_pred, y_train_torch)
    if t % 100 == 99:
        print(t, loss.item())

    # Zero the gradients before running the backward/training pass.
    optimizer.zero_grad()
    model.zero_grad()

    # Backward pass: compute gradient of the loss with respect to all the learnable
    # parameters of the model. Internally, the parameters of each Module are stored
    # in Tensors with requires_grad=True, so this call will compute gradients for
    # all learnable parameters in the model.
    loss.backward()
    optimizer.step()

    # Update the weights using gradient descent. Each parameter is a Tensor, so
    # we can access its gradients like we did before.
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad


99 7296.1962890625
199 7216.1279296875
299 7162.02880859375
399 7143.966796875
499 7132.01953125
599 7123.76220703125
699 7117.7119140625
799 7112.9736328125
899 7109.189453125
999 7106.0634765625


### Test Error

In [18]:
y_pred_test = model(x_test_torch)
loss_fn(y_test_torch, y_pred_test)

/Users/andrewgonsalves/opt/miniconda3/lib/python3.7/site-packages/torch/nn/modules/loss.py:431: UserWarning: Using a target size (torch.Size([401, 1])) that is different to the input size (torch.Size([401])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


tensor(4.6999, grad_fn=<SumBackward0>)

### RSS

In [19]:
RSS = ((model(x_train_torch)-y_train_torch)**2).sum()
print(RSS)

tensor(7106.0352, grad_fn=<SumBackward0>)
